In [1]:
#######################################
## This code takes speeches from the ##
## Members of the Federal Reserve and##
##  turns the PDFs into text files.  ##
#######################################

#Import packages
import os #To change working directories and things of that nature.
import csv #To read and write csv files.
import re #For regular expressions. 
import nltk #For text manipulation and things of that nature. 
import gensim #We need the parser in here. 
import pandas #Panda. Panda. Panda panda panda panda!!  
import numpy #For mathemagic.
import copy #To copy lists.
import collections #For counting and dictionaries and things of that nature.
import textract 
#Set working directory. 
os.chdir('C:\Users\jeann\OneDrive\Documents\DissertationData\Fed Board of Governors') 

##Get all files in folder
folders = os.listdir("C:\Users\jeann\OneDrive\Documents\DissertationData\Fed Board of Governors")


In [2]:
############################### 
########## Functions ##########
###############################

##This is code I found online. 
##Credit where credit is due. 
##https://gist.github.com/benhoyt/dfafeab26d7c02a52ed17b6229f0cb52
import sys
import time


def tokenize(string): #Jean changed this. 
    """Convert string to lowercase and split into words (ignoring
    punctuation), returning list of words.
    """
    new = string.split(' ')
    return [x.lower() for x in new]
    #return re.findall(r'\w+', string.lower())


def count_ngrams(lines, min_length=2, max_length=4):
    """Iterate through given lines iterator (file object or list of
    lines) and return n-gram frequencies. The return value is a dict
    mapping the length of the n-gram to a collections.Counter
    object of n-gram tuple and number of times that n-gram occurred.
    Returned dict includes n-grams of length min_length to max_length.
    """
    lengths = range(min_length, max_length + 1)
    ngrams = {length: collections.Counter() for length in lengths}
    queue = collections.deque(maxlen=max_length)

    # Helper function to add n-grams at start of current queue to dict
    def add_queue():
        current = tuple(queue)
        for length in lengths:
            if len(current) >= length:
                ngrams[length][current[:length]] += 1

    # Loop through all lines and words and add n-grams to dict
    for line in lines:
        for word in tokenize(line):         #CHANGE THIS Line. Want to keep punctuation. 
            queue.append(word)
            if len(queue) >= max_length:
                add_queue()

    # Make sure we get the n-grams at the tail end of the queue
    while len(queue) > min_length:
        queue.popleft()
        add_queue()

    return ngrams


def print_most_frequent(ngrams, num=10):
    """Print num most common n-grams of each length in n-grams dict."""
    for n in sorted(ngrams):
        print('----- {} most common {}-grams -----'.format(num, n))
        for gram, count in ngrams[n].most_common(num):
            print('{0}: {1}'.format(' '.join(gram), count))
        print('') 

#My code.
def multiwordReplace(text, wordDic):
    """
    Take a text and replace words that match a key in a dictionary with
    the associated value. Return the changed text.
    """
    keys = wordDic.keys()
    for key in keys:
        text = re.sub(pattern = key, repl = wordDic.get(key), string = text, flags = re.I)
    return text 


In [3]:
#######################
#Convert PDFs to text.#
####################### 

#Create empyt list. Append text, name of file (speech), and speaker info (folder).
allSpeeches = [] 

for folder in folders:
    os.chdir(('C:\Users\jeann\OneDrive\Documents\DissertationData\Fed Board of Governors' + "\\" + folder)) 
    govner = os.listdir(('C:\Users\jeann\OneDrive\Documents\DissertationData\Fed Board of Governors' + "\\" + folder)) 
    speeches = [[textract.process((speech), method = 'pdftotext'), speech, folder]
                for speech in govner] 
    allSpeeches.append(speeches)
        

In [4]:
#We have a list of lists of lists.  A list of governers-lists.
#Every governer-list has a speech, file name, and folder name. 
#Want to turn it into just a list of lists. 
#A list were every element is a list containing a speech, file name, and folder name. 

allSpeeches = [data for folder in allSpeeches for data in folder] 

In [5]:
#Create a data frame.  
#We will eventually export this data frame to 
#R. 
data = pandas.DataFrame(allSpeeches, columns = ["text","name","gov"]) 

In [51]:
#Let's create a new variable that holds cleaned and processed data.
#Make sure to keep the original text. 

#Remove non-ASCII characters.  Older documents are often misread. 
data['clean'] = [re.sub(r'[^\x00-\x7F]+','', speech) for speech in data['text']] 

#Remove all extra white spaces. This is more of an issue for
#older texts than for more recent documents.
data['clean'] = [re.sub('\s+', ' ',speech) for speech in data['clean']] 

In [14]:
#Want to find most common n-grans.  To do this, turn all text lowercase and change new ling
#characters to spaces.  Do this because the text has a lot of new line characters 
#because the text is old. 
#oneString = [speech[0].replace('\r\n', ' ') for speech in allSpeeches] 
oneString = data['clean'].tolist() 
oneString = ' '.join(oneString)

In [15]:
#Tokenize words by space.  
tokes = oneString.split()  

In [16]:
#Find n-grams for n 4:15. 
ngrams = count_ngrams(tokes, min_length = 2, max_length = 5) 

In [1]:
#Go through top 250 n-grams of a given length
#to decide which ones should be made into n-grams
#and which ones should be left as is.  
for gram, count in ngrams[2].most_common(500):
            print('{0}: {1}'.format(' '.join(gram), count))

In [52]:
#######################################
### Find common ngrams and replace. ###
### The first round is to replace   ###
### words that are separated due to ###
###   being read improperly.  This  ###
### mostly affects older documents. ###
#######################################

#Set rule to decide which terms to include.  
#Rule: 1.5 x the minimum number of articles a word
#needs to appear in in order to be included 
#in the anlaysis.  Currently 40 documents, 
#so an n-gram has to appear at least 60
#times to be replaced.
#NOTE: This is only for terms read incorrectly due to being historical documents. 
fifteenKey  = ['c o m m e r c i a l b a n k s','c h a r a c t e r i s t i c s',
               'p e r c e n t a g e p o i n t','s a v i n g s d e p o s i t s',
               'b o a r d of g o v e r n o r s','f o r e i g n b r a n c h e s'] 
fifteenVal  = ['commercial banks','characteristics',
               'percentage point','savings deposits',
               'board of governors','foreign branches']

fourteenKey = ['f e d e r a l r e s e r v e','m o n e t a r y p o l i c y','r e p r e s e n t a t i v e',
               'c o m m e r c i a l b a n k','a d m i n i s t r a t i o n','r e s p o n s i b i l i t y',
               'a s s o c i a t e d w i t h','r e c o m m e n d a t i o n','m u l t i - n a t i o n a l'] 
fourteenVal = ['federal reserve','monetary policy','representative',
               'commercial bank','administration','responsibility',
               'associated with','recommendation','multi-national']

thirteenKey = ['i n t e r e s t r a t e s','the u n i t e d s t a t e s','s u b s t a n t i a l l y',
               'i n t e r n a t i o n a l','p a r t i c i p a t i o n','o p p o r t u n i t i e s',
               'f o u r t h q u a r t e r','s i g n i f i c a n t l y','a p p r o x i m a t e l y',
               'p r o p o r t i o n a t e','c i r c u m s t a n c e s','o f governors o f the f e d e r a l',
               't o t a l d e p o s i t s','o r g a n i z a t i o n s','r e l a t i o n s h i p s',
               'c o n s i d e r a t i o n','m a n u f a c t u r i n g'] 
thirteenVal = ['interest rates','the united states','substantially',
               'international','participation','opportunities',
               'fourth quarter','significantly','approximately',
               'proportionate','circumstances','of governors of the federal',
               'total deposits','organizations','relationships',
               'consideration','manufacturing'] 

twelveKey   = ['i n s t i t u t i o n s','u n i t e d s t a t e s','i n t e r e s t r a t e',
               'p a r t i c u l a r l y','d i s t r i b u t i o n','r e q u i r e m e n t s',
               'i n f l a t i o n a r y','u n e m p l o y m e n t','p r o d u c t i v i t y',
               'f o r e i g n a s s e t','a v a i l a b i l i t y','e x p e n d i t u r e s',
               'c o r p o r a t i o n s','c e r t i f i c a t e s','c o n s i d e r a b l y',
               'd i f f e r e n t i a l','d e v e l o p m e n t s','o r g a n i z a t i o n',
               'e u r o - d o l l a r s','r e l a t i o n s h i p','c o n s t r u c t i o n',
               'o c c u p a t i o n a l','i n c r e a s i n g l y','p r o f e s s i o n a l',
               'c o n s i d e r a b l e'] 
twelveVal   = ['institutions','united states','interest rate',
               'particularly','distribution','requirements',
               'inflationary','umemployment','productivity',
               'foreign assets','availability','expenditures',
               'corporations','certificates','considerably',
               'differential','developments','organization',
               'euro-dollars','relationship','construction',
               'occupational','increasingly','professional',
               'considerable']  

elevenKey   = ['i n s t i t u t i o n','s u b s t a n t i a l','e u r o - d o l l a r',
               'l i a b i l i t i e s','o u t s t a n d i n g','s i g n i f i c a n t',
               'r e q u i r e m e n t','s t a t e and l o c a l','d e v e l o p m e n t',
               'a s s o c i a t i o n','i n f o r m a t i o n','c o r p o r a t i o n',
               'r e p r e s e n t e d','i n v e s t m e n t s','d e r i v a t i v e s',
               'e x p e n d i t u r e','o p p o r t u n i t y','a p p r o p r i a t e',
               'e s t a b l i s h e d','s t a t i s t i c a l','w i t h r e s p e c t',
               'o b l i g a t i o n s','d i f f e r e n c e s','t r a n s a c t i o n',
               'a p p r o x i m a t e','c o m p e t i t i o n','u n s o l i c i t e d',
               'p e r f o r m a n c e','a u t h o r i t i e s','e x p e c t a t i o n',
               'r a t e c e i l i n g','e n v i r o n m e n t','c o n c e n t r a t e',
               'e x p e r i e n c e d','t r a d i t i o n a l','p r e p a r a t i o n',
               'i n s t r u m e n t s','e x a m i n a t i o n','l e g i s l a t i o n',
               'a g r i c u l t u r e','i c r e a s i n g l y','i m m i g r a t i o n',
               'c o m p e t i t i v e','a l t e r n a t i v e'] 
elevenVal   = ['institution','substantial','euro-dollar',
               'liabilities','outstanding','significant',
               'requirement','state and local','development',
               'association','information','corporation',
               'represented','investments','derivatives',
               'expenditure','opportunity','appropriate',
               'established','statistical','with respect',
               'obligations','differences','transaction',
               'approximate','competition','unsolicited',
               'performance','authorities','expectation',
               'rate ceiling','environment','concentrate',
               'experienced','traditional','preparation',
               'instruments','examination','legislation',
               'agriculture','increasingly','immigration',
               'competitive','alternative'] 

tenKey      = ['s e c u r i t i e s','e m p l o y m e n t','p a r t i c u l a r','p r o p o r t i o n',
               'c o m m e r c i a l','g o v e r n m e n t','i n v e s t m e n t','e x p e r i e n c e',
               'i n c r e a s i n g','i n c r e a s e i n','i n d i v i d u a l','r e l a t i v e l y',
               'e s p e c i a l l y','p o p u l a t i o n','r e g u l a t i o n','d i f f e r e n c e',
               'a c t i v i t i e s','s h o r t - t e r m','i n d u s t r i a l','b l a c k b a n k s',
               'p r o j e c t i o n','a d d i t i o n a l','c o n d i t i o n s','d e r i v a t i v e',
               's t a t i s t i c s','o c c u p a t i o n','i n d u s t r i e s','b u s i n e s s e s',
               'o b l i g a t i o n','p r o d u c t i o n','monetary r e s t r a i n t','b a l a n c e o f payments',
               'r e s t r i c t e d','c r e d i t c a r d','m a n a g e m e n t','d e v e l o p i n g',
               'i n s t r u m e n t','a n n u a l r a t e','c o m m i s s i o n','r e p o r t i n g banks',
               't o t a l l o a n s','c o n s t i t u t e','l a b o r f o r c e','a s s o c i a t e d',
               'o b j e c t i v e s','p e r s o n a l l y','a f f i l i a t e s','m o n e t a r i s t',
               'a g g r e g a t e s','a d j u s t m e n t','weekly r e p o r t i n g','c o n t i n u i n g',
               'r e g i s t e r e d','c o n c l u s i o n','a p p a r e n t l y','s e a s o n a l l y',
               'c a t e g o r i e s','e n t e r p r i s e','s u f f i c i e n t','the p r i n c i p a l',
               'i n c r e a s e o f','p r o f e s s i o n','r e f l e c t i n g','w a s h i n g t o n',
               'f o r e i g n e r s','r e p a r a t i o n','the c o m m i t t e e','d i s c u s s i o n',
               'a c c e l e r a t e','r e g u l a t o r y','s t r e n g t h e n','a c c o u n t i n g',
               's u p e r v i s o r','p e r c e n t a g e','p r o d u c t i v e','g u i d e l i n e s',
               'r e a s o n a b l e','r e a s o n a b l y','s e a s o n a l l y','r e f l e c t i n g',
               'u n e m p l o y e d','c a p i t a l i s t','c a p i t a l i s m','a n t i c i p a t e'] 
tenVal      = ['securities','employment','particular','proportion',
               'commercial','government','investment','experience',
               'increasing','increase in','individual','relatively',
               'especially','population','regulation','difference',
               'activities','short-term','industrial','black banks',
               'projection','additional','conditions','derivative',
               'statistics','occupation','industries','businesses',
               'obligation','production','monetary restraint','balance of payments',
               'restricted','credit card','management','developing',
               'instrument','annual rate','commission','reporting banks',
               'total loans','constitute','labor force','associated',
               'objectives','personally','affiliates','monetarist',
               'aggregates','adjustment','weekly reporting','continuing',
               'registered','conclusion','apparently','seasonally',
               'categories','enterprise','sufficient','the principal',
               'increase of','profession','reflecting','washington',
               'foreigners','reparation','the committee','discussion',
               'accelerate','regulatory','strengthen','accounting',
               'supervisor','percentage','productive','guidelines',
               'reasonable','reasonably','seasonally','reflecting',
               'unemployed','capitalist','capitalism','anticipate'] 

nineKey     = ['i n f l a t i o n','f i n a n c i a l','r e s t r a i n t','d i f f e r e n t',
               'r e p r e s e n t','i n c r e a s e d','c o u n t r i e s','b o r r o w i n g'
               'a b i l i t i e s','a v a i l a b l e','e x p a n s i o n','f i n a n c i n g',
               's t a t i s t i c','i m p o r t a n t','e d u c a t i o n','r e p o r t i n g',
               'p e r c e n t o f','c o m m i t t e e','p r e s e n t e d','d i f f i c u l t',
               's i t u a t i o n','e s t a b l i s h','g e n e r a l l y','l o n g - t e r m',
               'f o r m a t i o n','c o n d i t i o n','p o t e n t i a l','o b j e c t i v e',
               'l i q u i d i t y','e f f e c t i v e','o p e r a t i o n','r e s o u r c e s',
               's t r u c t u r e','p r i n c i p a l','b l a c k b a n k','g u i d e l i n e',
               'b a l a n c e o f','s t a b i l i t y','a u t h o r i t y','c o m m u n i t y',
               'c o r p o r a t e','a g g r e g a t e','i n c l u d i n g','p r o v i s i o n',
               'n e c e s s a r y','p r e s s u r e s','i n d i c a t e d','p r i m a r i l y',
               'p o s i t i o n s','a t t r i t i o n','m i g r a t i o n','r e c o m m e n d',
               'r e f l e c t e d','a f f i l i a t e','p r o j e c t e d','i n f l u e n c e',
               'l i a b i l i t y','v i r t u a l l y','e s s e n t i a l','r e d u c t i o n',
               'f o l l o w i n g','r e g u l a t o r','a s s o c i a t e','o n e - t h i r d',
               'c o m p a n i e s','p o l l u t i o n','o p e r a t i n g','m o n e t a r y policy',
               'p r e s i d e n t','b o r r o w e r s','e s t i m a t e s','c o n s u m e r s',
               'n o n w h i t e s','r e c e s s i o n','p r a c t i c e s','s o l i c i t e d',
               'b e g i n n i n g','o f f i c i a l s','a b i l i t y o f','r e c o g n i z e',
               'p o r t f o l i o','d e v e l o p e d','i n v e s t o r s','t e c h n i c a l',
               'f o r e i g n e r','g u a r a n t e e','t h e r e f o r e','m e n t i o n e d',
               'e f f e c t s o f','p a t t e r n o f','a c c o u n t e d','i n s u r a n c e',
               'h o u s e h o l d','q u e s t i o n s','f o r w h i t e s','g e n e r a l l y',
               'a s s i s t a n t','r e c o m m e n d','a u t h o r i z e','s h o r t - r u n',
               'm i n o r i t i e s','u n c e r t a i n','o n e - f i f t h','v o l u n t a r y']  
nineVal     = ['inflation','financial','restraint','different',
               'represent','increased','countries','borrowing'
               'abilities','available','expansion','financing',
               'statistic','important','education','reporting',
               'percent of','committee','presented','difficult',
               'situation','establish','generally','long-term',
               'formation','condition','potential','objective',
               'liquidity','effective','operation','resources',
               'structure','principal','black bank','guideline',
               'balance of','stability','authority','c o m m u n i t y',
               'corporate','aggregate','including','provision',
               'necessary','pressures','indicated','primarily',
               'positions','attrition','migration','recommend',
               'reflected','affiliate','projected','influence',
               'liability','virtually','essential','reduction',
               'following','regulator','associate','one-third',
               'companies','pollution','operating','monetary policy',
               'president','borrowers','estimates','consumers',
               'nonwhites','recession','practices','solicited',
               'beginning','officials','ability of','recognize',
               'portfolio','developed','investors','technical',
               'foreigner','guarantee','therefore','mentioned',
               'effects of','pattern of','accounted','insurance',
               'household','questions','for whites','generally',
               'assistant','recommend','authorize','short-run',
               'minorities','uncertain','one-fifth','voluntary']  

eightKey    = ['i n c r e a s e','d e p o s i t s','n a t i o n a l','f e d e r a l reserve','i n t e r e s t',
               'm o n e t a r y','b u s i n e s s','c o n s i d e r','f a m i l i e s','e c o n o m i c',
               'p o s i t i o n','m o r t g a g e','a d d i t i o n','c o n s u m e r','s e r v i c e s',
               'o f f i c i a l','p r e s s u r e','c o n t i n u e','a n a l y s i s','p r o g r e s s',
               'r e s t r i c t','p e r s o n a l','d e c l i n e d','q u e s t i o n','b r a n c h e s',
               'p o l i c i e s','a m e r i c a n','a l t h o u g h','a c t i v i t y','l a s t y e a r',
               'm a i n t a i n','i n d u s t r y','r e p o r t e d','r e q u i r e d','p o s s i b l e',
               'p r o v i d e d','e x p e c t e d','g o v e r n o r','a d j u s t e d','a c c o u n t s',
               'r e l a t i o n','t h e b o a r d','p r o b a b l y','d e l i v e r y','m o r e o v e r',
               'p r o b l e m s','p r a c t i c e','m o d e r a t e','d e p o s i t s','demand d e p o s i t',
               'c o n g r e s s','f e b r u a r y','r e c e i v e d','p r o s p e c t','i n the f i r s t',
               'o c c u r r e d','f o r w h i t e','p r e v i o u s','s l i g h t l y','c o n t r a s t',
               'r e c o v e r y','p a y m e n t s','c h a n g e s in','b o r r o w e r','r e c o r d e d',
               'd e c i s i o n','s t r e n g t h','s p e c i f i c','a p p r o a c h','e x i s t i n g',
               'r e c e n t l y','e d u c t i o n','p o r t i o n of','d i s t r i c t','d i r e c t l y',
               'c a p a c i t y','t h o u s a n d','b e n e f i t s','s e c u r i t y','e m p l o y e d',
               'd o m e s t i c','a g e n c i e s','s e a s o n a l','the p r i v a t e','d i s c o u n t',
               'c u s t o m e r','d e p o s i t s','p u r c h a s e','m e a s u r e s','p r o g r a m s',
               's t u d e n t s','f e d e r a l funds','a p p a r e n t','s e l e c t e d','c o m p a r e d',
               'p r o p o s a l','s t a n d a r d','f e d e r a l government','i n s t a n c e','i n v e s t o r',
               'e v i d e n c e','m a r g i n a l','c h i l d r e n','a c t u a l l y','c u r r e n c y',
               'r e s p o n s e','c a t e g o r y','e a r n i n g s','shown i n t a b l e','the e f f e c t s',
               'e m p l o y e e','m a t u r i t y','s e p a r a t e','f a c t t h a t','f u n c t i o n',
               'c o u n t i n g','c u l t u r a l','l o n g - r u n','bank c r e d i t card','p a t t e r n s',
               'r e s e r v e requirements','m i n o r i t y','s e l e c t e d','c i v i l i a n','r e l a t i v e',
               'e s t i m a t e','t r e a s u r y','r e s i d e n t','b e h a v i o r','c h a i r m a n']  
eightVal    = ['increase','deposits','national','federal reserve','interest',
               'monetary','business','consider','families','economic',
               'position','mortgage','addition','consumer','services',
               'official','pressure','continue','analysis','progress',
               'restrict','personal','declined','question','branches',
               'policies','american','although','activity','last year',
               'maintain','industry','reported','required','possible',
               'provided','expected','governor','adjusted','accounts',
               'relation','the board','probably','delivery','moreover',
               'problems','practice','moderate','deposits','demand deposit',
               'congress','february','received','prospect','in the first',
               'occurred','for white','previous','slightly','contrast',
               'recovery','payments','changes in','borrower','recorded',
               'decision','strength','specific','approach','existing',
               'recently','education','portion of','district','directly',
               'capacity','thousand','benefits','security','employed',
               'domestic','agencies','seasonal','the private','discount',
               'customer','deposits','purchase','measures','programs',
               'students','federal funds','apparent','selected','compared',
               'proposal','standard','federal government','instance','investor',
               'evidence','marginal','children','actually','currency',
               'response','category','earnings','shown in table','the effects',
               'employee','maturity','separate','fact that','function',
               'counting','cultural','long-run','bank credit card','patterns',
               'reserve requirements','minority','selected','civilian','relative',
               'estimate','treasury','resident','behavior','chairman']  

sevenKey    = ['f e d e r a l','r e s e r v e','b i l l i o n','p e r c e n t','d e p o s i t',
               'm i l l i o n','f o r e i g n','c a p i t a l','p r e s e n t','d e c l i n e',
               'a b i l i t y','r e q u i r e','g e n e r a l','c e i l i n g','p o r t i o n',
               'q u a r t e r','a c c o u n t','p r i v a t e','c u r r e n t','b a l a n c e',
               'd o l l a r s','p r o v i d e','s e r v i c e','p r o d u c t','d e v e l o p',
               's a v i n g s','r e f l e c t','h o w e v e r','s p e c i a l','c e n t r a l',
               'f u r t h e r','b a n k i n g','e c o n o m y','m a r k e t s','c o u n t r y',
               'p r o g r a m','g r e a t e r','l e n d i n g','f i n a n c e','a m e r i c a',
               't h r o u g h','s o u r c e s','p a t t e r n','a g a i n s t','h o l d i n g',
               'p r o b l e m','r e s u l t s','b e t w e e n','b e l i e v e','e f f e c t s',
               'w o r k e r s','s i z a b l e','m e a s u r e','p r o c e s s','e a r l i e r',
               'c o l l e g e','c h a n g e s','c e r t a i n','c l e a r l y','s u b j e c t',
               'e f f o r t s','f i g u r e s','e x p o r t s','h o u s i n g','o v e r a l l',
               'monetary p o l i c y','c o n c e r n','b e c a u s e','r a p i d l y','n e g r o e s',
               'w e l f a r e','p a y m e n t','b e n e f i t','c r e d i t card','p e r i o d s',
               'a c t i o n s','o u t f l o w','q u a l i t y','r e c o v e r','e x a m p l e',
               'd i s c u s s','l ia b il it ie s','s e c t o r s','r e l a t e d','s u r p l u s',
               'r e l e a s e','e a r n i n g','c o l l e c t','p e r f o r m','r o u g h l y',
               'j a n u a r y','a t t r a c t','m i s s i o n','a n o t h e r','r e p o r t s',
               's h a r p l y','m e m b e r s','h e a v i l y','o u t l o o k','p r o t e c t',
               'c o n s i s t','w h e t h e r','s u c c e s s','p u r p o s e','b o a r d \' s',
               'e v i d e n t','f a c t o r s','g r e a t l y','s i m i l a r','s e c t i o n',
               'c i t i z e n','v i s i o n s','l i m i t e d','a t t e m p t','i n v o l v e',
               'o b v i o u s','s t u d e n t','e l e c t e d','s o l i c i t','r e d u c e d',
               'm e n t i o n','a d v a n c e','a p p e a r s','p e n d i n g','c o u n t e d',
               'c o m p a n y','p r o d u c e','h i g h e s t','t y p i c a l','w i l l i n g',
               'b a n k e r s','a v e r a g e','s e v e r a l','c o n t r o l','d e f i c i t',
               'i n c l u d e','s u g g e s t','s e r i o u s','a n n u a l','o b t a i n'] 
sevenVal    = ['federal','reserve','billion','percent','deposit',
               'million','foreign','capital','present','decline',
               'ability','require','general','ceiling','portion',
               'quarter','account','private','current','balance',
               'dollars','provide','service','product','develop',
               'savings','reflect','however','special','central',
               'further','banking','economy','markets','country',
               'program','greater','lending','finance','america',
               'through','sources','pattern','against','holding',
               'problem','results','between','believe','effects',
               'workers','sizable','measure','process','earlier',
               'college','changes','certain','clearly','subject',
               'efforts','figures','exports','housing','overall',
               'monetary policy','concern','because','rapidly','negroes',
               'welfare','payment','benefit','credit card','periods',
               'actions','outflow','quality','recover','example',
               'discuss','liabilities','sectors','related','surplus',
               'release','earning','collect','perform','roughly',
               'january','attract','mission','another','reports',
               'sharply','members','heavily','outlook','protect',
               'consist','whether','success','purpose','board\'s',
               'evident','factors','greatly','similar','section',
               'citizen','visions','limited','attempt','involve',
               'obvious','student','elected','solicit','reduced',
               'mention','advance','appears','pending','counted',
               'company','produce','highest','typical','willing',
               'bankers','average','several','control','deficit',
               'include','suggest','serious','annual','obtain']  

sixKey      = ['c r e d i t','n a t i o n','m a r k e t','d o l l a r','d u r i n g','p e r i o d','p o l i c y',
               'e m p l o y','e f f e c t','s t a t e s','g o v e r n','r e s u l t','g r o w t h','p u b l i c',
               'u n i t e d','e x p o r t','s y s t e m','a s s e t s','b o r r o w','i n v e s t','r e c e n t',
               'd i r e c t','s h o u l d','s a v i n g','c h a n g e','i m p o r t','p r i c e s','b l a c k s',
               'h i g h e r','w h i t e s','e x p e c t','e x p o r t','m e m b e r','p e r s o n','b e f o r e',
               'b r a n c h','f i s c a l','t h o u g h','n u m b e r','l i q u i d','f i g u r e','l i t t l e',
               'a s s i s t','per c e n t','i n c o m e','r e a s o n','r i s i n g','d e m a n d','w o r k e r',
               'r a t h e r','o u t p u t','r a t i n g','f o l l o w','o b j e c t','a m o u n t','a p p e a r',
               'o t h e r s','a c t u a l','l e v e l s','m o n t h s','r e d u c e','p e o p l e','y i e l d s',
               's e c o n d','c o u r s e','s t r o n g','n e a r l y','o f f i c e','e f f o r t','f u t u r e',
               'r e m a i n','s e c t o r','r a i s e d']  
sixVal      = ['credit','nation','market','dollar','during','period','policy',
               'employ','effect','states','govern','result','growth','public',
               'united','export','system','assets','borrow','invest','recent',
               'direct','should','saving','change','import','prices','blacks',
               'higher','whites','expect','export','member','person','before',
               'branch','fiscal','though','number','liquid','figure','little',
               'assist','percent','income','reason','rising','demand','worker',
               'rather','output','rating','follow','object','amount','appear',
               'others','actual','levels','months','reduce','people','yields',
               'second','course','strong','nearly','office','effort','future',
               'remain','sector','raised']  

fiveKey     = ['t o t a l','b a n k s','w h i c h','c o u n t','b l a c k','l a r g e','per c e n t',
               'f i r s t','t h e s e','b o a r d','w h i l e','p r i c e','g r e a t','a s s e t',
               'l o a n s','a b o u t','s m a l l','g a i n s','m i g h t','u n i t y','n e g r o',
               't r e n d','t h o s e','w o r l d','b r o a d','c a u s e','p l a c e','g u i d e',
               'annual r a t e','a p r i l','l e a s e','b a s i c','t e r m s','f u l l y','f l o w s',
               'c o u r s e','p e r c e n t','y e a r s','w h i t e','s t a t e','w o u l d','i s s u e','t h e i r',
               'o t h e r','w o u l d','t h e r e', 'o f t h e']  
fiveVal     = ['total','banks','which','count','black','large','percent',
               'first','these','board','while','price','great','asset',
               'loans','about','small','gains','might','unity','negro',
               'trend','those','world','broad','cause','place','guide',
               'annual rate','april','lease','basic','terms','fully','flows',
               'course','percent','years','would','issue','their',
               'other','would','there','of the']

fourKey     = ['t h a t','c e n t','w i t h','r i s e','s u r e','l o a n','t h a n',
               'f a c t','b e e n','g a i n','g r o w','w o r k','f u n d','r e a d',
               'r o s e','l o o k','e a c h','m a d e','c a s e','per cent',
               'w h i t e','s t a t e','w i l l','y e a r','of t h e','h i g h']  
fourVal     = ['that','cent','with','rise','sure','loan','than',
               'fact','been','gain','grow','work','fund','read',
               'rose','look','each','made','case','percent',
               'white','state','will','year','of the','high']




#Create dictionaries for all of these. 
#dictionary = dict(zip(keys, values))
fifteen = dict(zip(fifteenKey, fifteenVal))
fourteen = dict(zip(fourteenKey, fourteenVal))
thirteen = dict(zip(thirteenKey, thirteenVal))
twelve = dict(zip(twelveKey, twelveVal)) 
eleven = dict(zip(elevenKey, elevenVal)) 
ten = dict(zip(tenKey, tenVal)) 
nine = dict(zip(nineKey, nineVal))
eight = dict(zip(eightKey, eightVal))
seven = dict(zip(sevenKey, sevenVal))
six = dict(zip(sixKey, sixVal))
five = dict(zip(fiveKey, fiveVal)) 
four = dict(zip(fourKey, fourVal))

In [53]:
#Go through each text and replace these
#ngrams.  
#IMPORTANT: Start with the longest ones first and move down.
data['clean'] = [multiwordReplace(speech, fifteen) for speech in data['clean']] 
data['clean'] = [multiwordReplace(speech, fourteen) for speech in data['clean']] 
data['clean'] = [multiwordReplace(speech, thirteen) for speech in data['clean']] 
data['clean'] = [multiwordReplace(speech, twelve) for speech in data['clean']] 
data['clean'] = [multiwordReplace(speech, eleven) for speech in data['clean']] 
data['clean'] = [multiwordReplace(speech, ten) for speech in data['clean']] 
data['clean'] = [multiwordReplace(speech, nine) for speech in data['clean']] 
data['clean'] = [multiwordReplace(speech, eight) for speech in data['clean']] 
data['clean'] = [multiwordReplace(speech, seven) for speech in data['clean']] 
data['clean'] = [multiwordReplace(speech, six) for speech in data['clean']] 
data['clean'] = [multiwordReplace(speech, five) for speech in data['clean']] 
data['clean'] = [multiwordReplace(speech, four) for speech in data['clean']] 


In [9]:
##Rerun the ngram analysis to catch any terms you missed previously. 
oneString = data['clean'].tolist() 
oneString = ' '.join(oneString) 
#Tokenize words by space.  
tokes = oneString.split() 

In [ ]:
#Boilerplat text
data['clean'] = [re.sub('digitized for fraser http://fraser\.stlouisfed\.org/ federal reserve bank of st. louis','',speech,flags = re.I)
                 for speech in data['clean']]
data['clean'] = [re.sub('home | news and events accessibility | contact us','',speech, flags = re.I) for speech in data['clean']]
data['clean'] = [re.sub('for release on delivery','',speech, flags = re.I) for speech in data['clean']]


#Institutions 
data['clean'] = [re.sub('board of governors of the federal reserve system','board_of_governors_of_the_federal_reserve_system',
                 speech, flags = re.I) for speech in data['clean']] 
data['clean'] = [re.sub('committee on banking, housing and urban affairs','committee_on_banking_housing_and_urban_affairs',
                        speech,flags = re.I) for speech in data['clean']] 
data['clean'] = [re.sub('committee on banking, finance and urban affairs','committee_on_banking_finance_and_urban_affairs',
                 speech, flags = re.I) for speech in data['clean']]
data['clean'] = [re.sub('federal open market committee','federal_open_market_committee',speech,flags = re.I) for speech in data['clean']]
data['clean'] = [re.sub('federal deposit insurance corporation','federal_deposit_insurance_corporation',speech,flags = re.I) for speech in data['clean']]
data['clean'] = [re.sub('board of governors','board_of_governors',speech,flags = re.I) for speech in data['clean']]
data['clean'] = [re.sub('federal reserve system','federal_reserve_system', speech,flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('federal reserve board','federal_reserve_board', speech,flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('federal reserve','federal_reserve', speech,flags=re.I) for speech in data['clean']]


#Districts
data['clean'] = [re.sub('federal reserve bank of st\. louis','federal_reserve_bank_of_st_louis',speech, flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('federal reserve bank of new york','federal_reserve_bank_of_new_york',speech, flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('federal reserve bank of kansas city','federal_reserve_bank_of_kansas_city',speech, flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('federal reserve bank of san francisco','federal_reserve_bank_of_san_francisco',speech, flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('federal reserve bank of boston','federal_reserve_bank_of_boston',speech, flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('federal reserve bank of philadelphia','federal_reserve_bank_of_philadelphia',speech, flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('federal reserve bank of richmond','federal_reserve_bank_of_richmond',speech, flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('federal reserve bank of cleveland','federal_reserve_bank_of_cleveland',speech, flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('federal reserve bank of atlanta','federal_reserve_bank_of_atlanta',speech, flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('federal reserve bank of chicago','federal_reserve_bank_of_chicago',speech, flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('federal reserve bank of minneapolis','federal_reserve_bank_of_minneapolis',speech, flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('federal reserve bank of dallas','federal_reserve_bank_of_dallas',speech, flags=re.I) for speech in data['clean']]

#Cities
data['clean'] = [re.sub('New York City','new_york',speech, flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('New York','new_york',speech, flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('Kansas City','kansas_city',speech, flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('San Francisco','san_francisco',speech, flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('St\. Louis','st_louis',speech, flags=re.I) for speech in data['clean']]


#Chairpersons
data['clean'] = [re.sub('paul a\. volcker','paul_a_volcker', speech,flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('ben s\. bernanke','ben_s_bernanke', speech,flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('arthur f\. burns','arthur_f_burns', speech,flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('g\. william miller','g_william_miller', speech,flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('thomas b\. mccabe','thomas_b_mccabe', speech,flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('william mcchesny martin','william_m_martin', speech,flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('alan greenspan','alan_greenspan', speech,flags=re.I) for speech in data['clean']]
data['clean'] = [re.sub('janet yellen','janet_yellen', speech,flags=re.I) for speech in data['clean']]


#Other terms  
data['clean'] = [re.sub('united states','united_states', speech,flags=re.I) for speech in data['clean']] 
data['clean'] = [re.sub('united states.','united_states', speech,flags=re.I) for speech in data['clean']] 
data['clean'] = [re.sub('bank holding companies','bank_holding_companies', speech,flags=re.I) for speech in data['clean']] 
data['clean'] = [re.sub('bank holding company','bank_holding_company', speech,flags=re.I) for speech in data['clean']] 
data['clean'] = [re.sub('goods and services','goods_and_services', speech,flags=re.I) for speech in data['clean']] 
data['clean'] = [re.sub('monetary policy','monetary_policy', speech,flags=re.I) for speech in data['clean']] 
data['clean'] = [re.sub('interest rates','interest_rates', speech,flags=re.I) for speech in data['clean']] 
data['clean'] = [re.sub('financial institutions','financial_institutions', speech,flags=re.I) for speech in data['clean']] 
data['clean'] = [re.sub('central banks','central_banks', speech,flags=re.I) for speech in data['clean']] 
data['clean'] = [re.sub('central bank','central_bank', speech,flags=re.I) for speech in data['clean']] 
data['clean'] = [re.sub('financial markets','financial_markets', speech,flags=re.I) for speech in data['clean']] 
data['clean'] = [re.sub('federal funds rate','federal_funds_rate', speech,flags=re.I) for speech in data['clean']] 
data['clean'] = [re.sub('federal funds','federal_funds', speech,flags=re.I) for speech in data['clean']] 
data['clean'] = [re.sub('financial system','financial_system', speech,flags=re.I) for speech in data['clean']] 
data['clean'] = [re.sub('real estate','real_estate', speech,flags=re.I) for speech in data['clean']] 
data['clean'] = [re.sub('commercial banks','commercial_banks', speech,flags=re.I) for speech in data['clean']] 
data['clean'] = [re.sub('monetary policy.','monetary_policy', speech,flags=re.I) for speech in data['clean']] 
data['clean'] = [re.sub('reserve banks','reserve_banks', speech,flags=re.I) for speech in data['clean']] 
data['clean'] = [re.sub('banking organizations','banking_organizations', speech,flags=re.I) for speech in data['clean']] 
data['clean'] = [re.sub('economic growth','economic_growth', speech,flags=re.I) for speech in data['clean']] 
data['clean'] = [re.sub('risk management','risk_management', speech,flags=re.I) for speech in data['clean']] 
data['clean'] = [re.sub('credit card','credit_card', speech,flags=re.I) for speech in data['clean']] 


In [30]:
#Save data frame as csv.
data.to_csv('C:\Users\jeann\OneDrive\Documents\DissertationData\\text.csv') 